In [1]:
# Creation of a Sentinel-2 Canadian Dataset
#
# Note: before running this code, create a new conda env named : gee
# 
# conda install -c anaconda pandas
# conda install -c conda-forge pysimplegui
# conda install openpyxl
# conda install -c conda-forge earthengine-api
# conda install -c conda-forge folium
#
# Then, authenticate yourself to Google Eeath Engine
# earthengine authenticate
# Inspiration: https://climada-python.readthedocs.io/en/stable/tutorial/climada_util_earth_engine.html
# and https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/sentinel-2-s2cloudless/index.ipynb#scrollTo=tCRB28UkJJjp 

# Amanda Boatswain Jacques / Etienne Lord
# Since 20 March 2021

In [2]:
from datetime import datetime
import pandas as pd
import folium
import webbrowser
import ee
import PySimpleGUI as sg
import sys
import os

#### RUN ME FIRST 

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

The following code was retrieved and tweaked from this official Earth Engine community tutorial: https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/sentinel-2-s2cloudless/index.ipynb#scrollTo=tCRB28UkJJjp 

### Define collection filter and cloud mask parameters

Define parameters that are used to filter the S2 image collection and determine cloud and cloud shadow identification.

|Parameter | Type | Description |
| :-- | :-- | :-- |
| `AOI` | `ee.Geometry` | Area of interest |
| `START_DATE` | string | Image collection start date (inclusive) |
| `END_DATE` | string | Image collection end date (exclusive) |
| `CLOUD_FILTER` | integer | Maximum image cloud cover percent allowed in image collection |
| `CLD_PRB_THRESH` | integer | Cloud probability (%); values greater than are considered cloud |
| `NIR_DRK_THRESH` | float | Near-infrared reflectance; values less than are considered potential cloud shadow |
| `CLD_PRJ_DIST` | float | Maximum distance (km) to search for cloud shadows from cloud edges |
| `BUFFER` | integer | Distance (m) to dilate the edge of cloud-identified objects |

In [4]:
# Set the thresholds that will be used to build the cloud/shadow mask 
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

## Useful Functions 

### Build a Sentinel-2 collection

Combine the Sentinel-2 Surface Reflectance and Cloud probability image collections using the assigned date and bounds 

Define a function to filter the SR and s2cloudless collections according to area of interest and date parameters, then join them on the `system:index` property. The result is a copy of the SR collection where each image has a new `'s2cloudless'` property whose value is the corresponding s2cloudless image.

In [5]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

### Cloud and Shadow Masking

#### Cloud components

Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.

In [6]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

#### Cloud shadow components

Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input. Note that the image input needs to be the result of the above `add_cloud_bands` function because it relies on knowing which pixels are considered cloudy (`'clouds'` band).

In [7]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs':'EPSG:3978', 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

#### Final cloud-shadow mask

Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.

In [8]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': 'EPSG:3978', 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img.addBands(is_cld_shdw)

### Define cloud mask application function

Define a function to apply the cloud mask to each image in the collection.

In [9]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

### Visualize the results

In [10]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

#### Create ROI  Square
Extract the region of interest (ROI) by creating a 64 x 64 pixel square around each point of interest. 

In [11]:
# Create a rectangle region using EPSG:3978, radius in meters, point is Lat, Long
def makeRectangle(point, xRadius=320, yRadius=320):
    proj = 'EPSG:3978';
    pointLatLon = ee.Geometry.Point(point);
    pointMeters = pointLatLon.transform(proj, 0.001);
    coords = pointMeters.coordinates();
    minX = ee.Number(coords.get(0)).subtract(xRadius);
    minY = ee.Number(coords.get(1)).subtract(yRadius);
    maxX = ee.Number(coords.get(0)).add(xRadius);
    maxY = ee.Number(coords.get(1)).add(yRadius);
    rect = ee.Geometry.Rectangle([minX, minY, maxX, maxY], proj, False);
    return rect;

In [12]:
# Select 5 points in the .csv sheet for a test 
points = [(0, "Ontario", "ON", "corn", 147, -77.32844954, 44.27797623),
          (1, "Ontario", "ON", "corn", 147, -77.37852316, 44.30095457),
          (2, "Quebec", "QC", "corn", 147, -75.00974578, 45.16622162),
          (3, "Quebec", "QC", "corn", 147, -74.99369555, 45.14719366), 
          (4, "Quebec", "QC", "corn", 147, -74.44633007, 45.22942979)]

In [13]:
# Create a list of seasons to iterate over 
time_intervals = [("2019-06-01","2019-06-30"), ("2019-07-01","2019-07-31"), 
                  ("2019-08-01","2019-08-31"), ("2019-09-01","2019-09-30"),
                  ("2019-10-01","2019-10-31")]

#### Create an image collection for a given point and all time intervals. 

#### Process the collection
Add the created masked and original layers to the final map for a point. Add cloud and cloud shadow component bands to each image and then apply the mask to each image. 

In [20]:
#center = [-74.44633007, 45.22942979]
m = folium.Map(zoom_start = 14)


for i, point in enumerate(points):
    # create the roi 
    ROI = makeRectangle([point[5], point[6]])
    print(point)
    for j, season in enumerate(time_intervals):
        print(season)
        # create an image collection for every point and month, mask clouds and shadows and then take the median image.  
        s2_sr_cloud_collection = get_s2_sr_cld_col(ROI, season[0], season[1])
        s2_sr_median = (s2_sr_cloud_collection.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
        
        # Add layers to the folium map.
        m.add_ee_layer(s2_sr_median,
                       {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                       ("Point: " + str(point[0]) + ", Province: " + point[1] + ", " + str(season)), True, 1, 9)
        
# center using the last evaluated ROI
center = ROI.centroid(10).coordinates().reverse().getInfo()
        
# Add a layer control panel to the map.
m.add_child(folium.LayerControl())     

display(m)

(0, 'Ontario', 'ON', 'corn', 147, -77.32844954, 44.27797623)
('2019-06-01', '2019-06-30')
('2019-07-01', '2019-07-31')
('2019-08-01', '2019-08-31')
('2019-09-01', '2019-09-30')
('2019-10-01', '2019-10-31')
(1, 'Ontario', 'ON', 'corn', 147, -77.37852316, 44.30095457)
('2019-06-01', '2019-06-30')
('2019-07-01', '2019-07-31')
('2019-08-01', '2019-08-31')
('2019-09-01', '2019-09-30')
('2019-10-01', '2019-10-31')
(2, 'Quebec', 'QC', 'corn', 147, -75.00974578, 45.16622162)
('2019-06-01', '2019-06-30')
('2019-07-01', '2019-07-31')
('2019-08-01', '2019-08-31')
('2019-09-01', '2019-09-30')
('2019-10-01', '2019-10-31')
(3, 'Quebec', 'QC', 'corn', 147, -74.99369555, 45.14719366)
('2019-06-01', '2019-06-30')
('2019-07-01', '2019-07-31')
('2019-08-01', '2019-08-31')
('2019-09-01', '2019-09-30')
('2019-10-01', '2019-10-31')
(4, 'Quebec', 'QC', 'corn', 147, -74.44633007, 45.22942979)
('2019-06-01', '2019-06-30')
('2019-07-01', '2019-07-31')
('2019-08-01', '2019-08-31')
('2019-09-01', '2019-09-30')
('

 Retry the previous code with this new filtering process: 

In [15]:
# create new obtain_image_sentinel function 
def obtain_image_sentinel2(time_range, area):
    s2_sr_cld_col = get_s2_sr_cld_col(area, time_range[0], time_range[1])
    s2_sr_median = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()
    return s2_sr_median

In [16]:
# Specify the correct encoding and region
def get_url(name, image, scale, region):
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region),
        'crs':'EPSG:3978'
        })
    return path

In [17]:
for i, point in enumerate(points):
    ROI = makeRectangle([point[5], point[6]])
    for j, season in enumerate(time_intervals):
        image = obtain_image_sentinel2(season, ROI)
        url = get_url("IMG_"+str(point[0]), image, 10, ROI)
        print("point: ", point)
        print("season: ", season)
        print("url: ", url)

point:  (0, 'Ontario', 'ON', 'corn', 147, -77.32844954, 44.27797623)
season:  ('2019-06-01', '2019-06-30')
url:  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5d92b9147c05d643af07688498d335d5-fd9862e3d89281ca8c223ea02ca4de72:getPixels
point:  (0, 'Ontario', 'ON', 'corn', 147, -77.32844954, 44.27797623)
season:  ('2019-07-01', '2019-07-31')
url:  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9c6edc197267845c7b95d65d54e3cc8f-9dd0fbe47e5996f302bd1820ce19cb41:getPixels
point:  (0, 'Ontario', 'ON', 'corn', 147, -77.32844954, 44.27797623)
season:  ('2019-08-01', '2019-08-31')
url:  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2efc3f8b7ff24f545755cd0a7fbfa703-ed8a9ce20bde2118871e92946c8b82c8:getPixels
point:  (0, 'Ontario', 'ON', 'corn', 147, -77.32844954, 44.27797623)
season:  ('2019-09-01', '2019-09-30')
url:  https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thu